In [ ]:
%%writefile agent.py
from kaggle_environments import make, evaluate
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation, Configuration, Action, row_col
from gym import spaces
import gym
import numpy as np
import random

last_step =[]
searched_list = []
def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
    result = []
    for i in (-1,+1):
        result.append(((x+i+7)%7, y))
        result.append((x, (y+i+11)%11))
    return result

def safe_pos(agent,posi_degree_1,posi_degree_2):
    if len(posi_degree_1)!= 0:
        step = compute_path_dir(agent, posi_degree_1[0])#safest action
    elif len(posi_degree_2)!= 0:
        step = compute_path_dir(agent, posi_degree_2[0])#risky action
    else:
        step = random.randint(1,4)#death
    return step
    
def get_dist(p1, p2):
    path0 = abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])
    path1 = 7-abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])
    path2 = abs(p1[0]-p2[0]) + 11-abs(p1[1]-p2[1])
    path3 = 7-abs(p1[0]-p2[0]) + 11-abs(p1[1]-p2[1])
    dis_lis = [path0,path1,path2,path3]
    dist = min(dis_lis)
    return dist
def compute_path_dir(agent, dest):#cords to action
    #covering X axis first
        if agent[0] != dest[0]:
            if agent[0] > dest[0] and abs(agent[0] - dest[0])==1:
                return 1
            if agent[0] < dest[0] and abs(agent[0] - dest[0])>1:
                return 1
            elif agent[0] < dest[0] and abs(agent[0] - dest[0])==1:
                return 3
            elif agent[0] > dest[0] and abs(agent[0] - dest[0])>1:
                return 3
        else:#Y axis
            if agent[1] > dest[1] and abs(agent[1] - dest[1])==1:
                return 4
            elif agent[1] < dest[1] and abs(agent[1] - dest[1])>1:
                return 4
            elif agent[1] < dest[1] and abs(agent[1] - dest[1])==1:
                return 2
            elif agent[1] > dest[1] and abs(agent[1] - dest[1])>1:
                return 2

def compute_path(agent, dest, posi):
    distance = []
    for i in posi:
        dist = get_dist(i,dest)
        distance.append(dist)
    final = posi[distance.index(min(distance))]
    return compute_path_dir(agent,final)

def compute_safety(agent, dest, posi):
    distance = []
    for i in posi:
        dist = get_dist(i,dest)
        distance.append(dist)
    final = posi[distance.index(min(distance))]
    return final
            
def food_dist(food_pos,agent,opponents,table,step):#finding closest food
    acc,dist = [],100 
    found = False
    for food in food_pos:
        opp_dist_list = []
        user_dist = get_dist(agent,food)
        for j in opponents:
            opp_dist_tem = get_dist(j,food)
            opp_dist_list.append(opp_dist_tem)
        opp_dist = int(min(opp_dist_list))    
        if opp_dist>user_dist and user_dist < dist:
            acc,dist = food,user_dist
            if step>40:
                searched_list = []
                found = get_path_safety(agent,acc,table,searched_list)
            else:
                found = True
    if len(acc) == 0:
        found = False
    return found, acc

def get_path_safety(p1,p2,table,searched_list):#recursive method for locating any blockades in the path
    posi = get_nearest_cells(p1[0],p1[1])
    posi = [i for i in posi if table[i[0],i[1]] not in [-3,-2] and list(i) not in searched_list]
    if len(posi) == 0:
        return False
    cord = compute_safety(p1, p2, posi)
    
    searched_list.append(list(cord))
    if list(cord) == p2:
        return True
    else:
        return get_path_safety(cord,p2,table,searched_list)
    
#def     
        
def create_table(obs_dict, config_dict):
    global last_step
    global prev_food
    observation = Observation(obs_dict)
    configuration = Configuration(config_dict)
    player_index = observation.index
    player_goose = observation.geese[player_index]
    player_head = player_goose[0]
    player_row, player_column = row_col(player_head, configuration.columns)
    
    user_head_val = 1
    user_body_val = -3
    opp_head_move_val = -1
    opp_head_val = -2
    food_val = 2
    forbidden = -5
    food_pos = []
    opp_head_pos = []
    table = np.zeros((7,11))
    for food in observation.food:
        x,y = row_col(food, configuration.columns)
        table[x,y] = food_val
        food_pos.append([x,y])
    # let's add all cells that are forbidden
    for i in range(4):
        goose = observation.geese[i]
        if len(goose) == 0:
            continue
            
        is_close_to_food = False
            
        
        x,y = row_col(goose[0], configuration.columns)
        possible_moves = get_nearest_cells(x,y) 
        
        if i != player_index:
            table[x,y] = opp_head_val
            opp_head_pos.append([x,y])# head can move anywhere
            for i1,i2 in possible_moves:
                if table[i1,i2] == food_val:
                    is_close_to_food = True
                if table[i1,i2] not in [-3,-2,-5,1]:
                    table[i1,i2] = opp_head_move_val 
        else:
            table[x,y] = user_head_val
            for i1,i2 in possible_moves:
                if table[x,y] == food_val:
                    is_close_to_food = True
        
        # usually we ignore the last tail cell but there are exceptions
        tail_change = -1
        if obs_dict['step'] % 40 == 0 and obs_dict['step']  >= 0:
            tail_change -= 1
        
        # we assume that the goose will eat the food
        if is_close_to_food:
            tail_change += 1
        if tail_change>=0:
            tail_change = None  

        for n in goose[1:tail_change]:
            x,y = row_col(n, configuration.columns)
            table[x,y] = -3 # forbidden cells
    
    if len(last_step) != 0 :
        table[last_step[0],last_step[1]] = forbidden
    x,y = row_col(player_head, configuration.columns)
    # add head position
    table[x,y] = user_head_val
    last_step = [x,y]
    return table, food_pos, opp_head_pos, player_goose


            
def make_move(board, food_pos,opp_head_pos,player_pos,obs_dict):
    player_head = player_pos[0]
    lenght = len(player_pos)
    legend = {
        1: 'NORTH',
        2: 'EAST',
        3: 'SOUTH',
        4: 'WEST'
    }
    step = None
    x,y = row_col(player_head, len(board[0]))
    posi = get_nearest_cells(x,y)
    posi_degree_1 = [i for i in posi if board[i[0],i[1]] in [0,2]] #safest path
    posi_degree_2 = [i for i in posi if board[i[0],i[1]] in [0,2,-1]] # risky
    center = [3,5]
    found, food = food_dist(food_pos,[x,y],opp_head_pos, board,obs_dict['step'])
    if len(posi_degree_1) == 0:
        step = safe_pos([x,y],posi_degree_1,posi_degree_2)
    elif found:
        step = compute_path([x,y], food,posi_degree_1)
    elif x == 3 and y == 5:
        step = compute_path_dir([x,y], posi_degree_1[0])
    elif x != 3 or y != 5:
        step = compute_path([x,y], center, posi_degree_1)
       
    return legend[step]

def agent(obs_dict, config_dict):
    board,food_pos,opp_head_pos,player_pos = create_table(obs_dict,config_dict)
    action = make_move(board,food_pos,opp_head_pos,player_pos,obs_dict)
    return action

In [ ]:
from kaggle_environments import make
env = make("hungry_geese", debug=True)

env.reset()
env.run(['agent.py', 'agent.py','agent.py','agent.py'])
#env.run(['agent.py', 'smart_agent.py','smart_agent.py','smart_agent.py'])
env.render(mode="ipython", width=500, height=450)